In [32]:
import pandas as pd
import plotly.express as px

# Quelles sont les communes qui alimentent les collèges de Rennes ?


In [33]:
carte_scolaire = pd.read_csv("./fr-en-carte-scolaire-colleges-publics.csv", sep=";")
carte_scolaire["Code_RNE"]

0       0352167R
1       0350052S
2       0351799R
3       0350970P
4       0350969N
          ...   
5411    0350916F
5412    0351638R
5413    0350916F
5414    0350895H
5415    0350965J
Name: Code_RNE, Length: 5416, dtype: object

In [34]:
colleges = pd.read_csv("./fr-en-adresse-et-geolocalisation-etablissements-premier-et-second-degre.csv", sep=";")
colleges.columns

Index(['Code établissement', 'Appellation officielle',
       'Dénomination principale', 'Patronyme uai', 'Secteur Public/Privé',
       'Adresse', 'Lieu dit', 'Boite postale', 'Code postal',
       'Localite d'acheminement', 'Commune', 'Coordonnee X', 'Coordonnee Y',
       'EPSG', 'Latitude', 'Longitude', 'Qualité d'appariement',
       'Localisation', 'Code nature', 'Nature', 'Code état établissement',
       'Etat établissement', 'Code département', 'Code région',
       'Code académie', 'Code commune', 'Département', 'Région', 'Académie',
       'Position', 'secteur_prive_code_type_contrat',
       'secteur_prive_libelle_type_contrat', 'code_ministere',
       'libelle_ministere', 'date_ouverture'],
      dtype='object')

In [35]:
# jointure sur le code rne
# on veut connaitre toute les villes qui alimentent les collèges de rennes
joined = carte_scolaire.join(colleges.set_index("Code établissement"), on=["Code_RNE"], how="left") 
joined[joined["Commune"] == "Rennes"].libelle_commune.unique()

array(['Chantepie', 'Rennes', 'Saint-Grégoire', 'Vezin-le-Coquet',
       'Thorigné-Fouillard'], dtype=object)

# Les effectifs dans les collèges de Rennes

In [36]:
# c'est un peu pénible, il faur piocher dans 2 jeux de données :(

new_df = pd.read_csv("fr-en-college-effectifs-niveau-sexe-lv.csv", sep=";")
old_df = pd.read_csv("fr-en-effectifs-second-degre.csv", sep=";")
annuaire = pd.read_csv("fr-en-annuaire-education.csv", sep=";")
# on ne sélectionne que les établissements des communes proches

effectifs_old = old_df[old_df["Type_d_etablissement"] == "COLLEGE"][["Annee_scolaire", "Nombre_d_eleves",  "Numero_d_etablissement"]]
effectifs_new = new_df[["Rentrée scolaire", "Numéro du collège", "Nombre d'élèves total (nombre d'élèves dans les formations du 1er cycle du 2nd degré et non du nombre total d'élèves inscrits dans l'établissement, les DIMA et les dispositifs-relais sont exclus)"]]

# on uniformise les noms de colonnes
effectifs_old.columns = ["Annee_scolaire", "effectifs", "identifiant"]
effectifs_old["rentrée"] = effectifs_old["Annee_scolaire"].str.split("-", expand=True)[0]
effectifs_old = effectifs_old[["rentrée", "effectifs", "identifiant"]]
effectifs_old = effectifs_old[effectifs_old["rentrée"] != "2019"]
effectifs_new.columns = ["rentrée", "identifiant", "effectifs"]

df_effectifs = pd.concat([effectifs_old, effectifs_new])
df_effectifs["rentrée"] = df_effectifs["rentrée"].astype("str")
df_effectifs["effectifs"] = df_effectifs["effectifs"].astype("int")

# jointure avec l'annuaire
df_effectifs = df_effectifs.join(annuaire.set_index("Identifiant_de_l_etablissement"), on="identifiant", how="inner")
df_effectifs = df_effectifs.sort_values(by="rentrée")

In [37]:
df_effectifs_rennes = df_effectifs[df_effectifs["Nom_commune"].str.match(r".*rennes.*", case=False)]
df_effectifs_rennes.groupby("rentrée")["effectifs"].sum()

rentrée
2015     9308
2016     9477
2017     9797
2018    10049
2019    10578
2020    10591
2021    10586
Name: effectifs, dtype: int64

In [41]:
df_effectifs_rennes_status = df_effectifs_rennes.groupby(["rentrée", "Statut_public_prive"])["effectifs"].sum().reset_index()
px.bar(df_effectifs_rennes_status, x="Statut_public_prive", y="effectifs", color="rentrée", barmode="group" ,title="Effectifs dans les collège de Rennes", width=800, height=600)

# Effectifs dans les écoles primaires alimentant Rennes

In [42]:
ecoles = pd.read_csv("fr-en-ecoles-effectifs-nb_classes.csv", sep=";")
ecoles["rentrée"] = ecoles["Rentrée scolaire"].astype(str)
ecoles = ecoles.sort_values("rentrée")

In [46]:

px.bar(ecoles[ecoles["Commune"] == "THORIGNE-FOUILLARD"], x="Dénomination principale", y="Nombre total d'élèves", color="rentrée", title="effectif 1er degré Thorigné", barmode="group")

In [52]:
ecoles[ecoles.Commune.str.startswith("CHAN")]

,Rentrée scolaire,Région académique,Académie,Département,Commune,Numéro de l'école,Dénomination principale,Patronyme,Secteur,REP,...,Nombre d'élèves en élémentaire hors ULIS,Nombre d'élèves en ULIS,Nombre d'élèves en CP hors ULIS,Nombre d'élèves en CE1 hors ULIS,Nombre d'élèves en CE2 hors ULIS,Nombre d'élèves en CM1 hors ULIS,Nombre d'élèves en CM2 hors ULIS,tri,Code Postal,rentrée
2104,2019,BRETAGNE,RENNES,ILLE-ET-VILAINE,CHANTEPIE,0352521A,ECOLE ELEMENTAIRE PUBLIQUE,LES DEUX RUISSEAUX,PUBLIC,0,...,134,0,22,24,39,32,17,81-BRETAGNE-RENNES-ILLE-ET-VILAINE-CHANTEPIE-0...,35135,2019
2103,2019,BRETAGNE,RENNES,ILLE-ET-VILAINE,CHANTEPIE,0352520Z,ECOLE PRIMAIRE PUBLIQUE,LES LANDES,PUBLIC,0,...,172,0,34,36,27,33,42,81-BRETAGNE-RENNES-ILLE-ET-VILAINE-CHANTEPIE-0...,35135,2019
2102,2019,BRETAGNE,RENNES,ILLE-ET-VILAINE,CHANTEPIE,0352159G,ECOLE MATERNELLE PUBLIQUE,LES DEUX RUISSEAUX,PUBLIC,0,...,0,0,0,0,0,0,0,81-BRETAGNE-RENNES-ILLE-ET-VILAINE-CHANTEPIE-0...,35135,2019
2101,2019,BRETAGNE,RENNES,ILLE-ET-VILAINE,CHANTEPIE,0351699G,ECOLE PRIMAIRE PRIVEE,ST JOSEPH,PRIVE,0,...,231,0,45,46,34,51,55,81-BRETAGNE-RENNES-ILLE-ET-VILAINE-CHANTEPIE-0...,35135,2019
2100,2019,BRETAGNE,RENNES,ILLE-ET-VILAINE,CHANTELOUP,0350168T,ECOLE PRIMAIRE PUBLIQUE,LUCIE AUBRAC,PUBLIC,0,...,147,0,31,21,32,32,31,81-BRETAGNE-RENNES-ILLE-ET-VILAINE-CHANTELOUP-...,35150,2019
1427,2020,BRETAGNE,RENNES,ILLE-ET-VILAINE,CHANTEPIE,0352521A,ECOLE ELEMENTAIRE PUBLIQUE,LES DEUX RUISSEAUX,PUBLIC,0,...,147,0,34,21,24,35,33,80-BRETAGNE-RENNES-ILLE-ET-VILAINE-CHANTEPIE-0...,35135,2020
1426,2020,BRETAGNE,RENNES,ILLE-ET-VILAINE,CHANTEPIE,0352520Z,ECOLE PRIMAIRE PUBLIQUE,LES LANDES,PUBLIC,0,...,189,0,45,41,38,27,38,80-BRETAGNE-RENNES-ILLE-ET-VILAINE-CHANTEPIE-0...,35135,2020
1425,2020,BRETAGNE,RENNES,ILLE-ET-VILAINE,CHANTEPIE,0352159G,ECOLE MATERNELLE PUBLIQUE,LES DEUX RUISSEAUX,PUBLIC,0,...,0,0,0,0,0,0,0,80-BRETAGNE-RENNES-ILLE-ET-VILAINE-CHANTEPIE-0...,35135,2020
1424,2020,BRETAGNE,RENNES,ILLE-ET-VILAINE,CHANTEPIE,0351699G,ECOLE PRIMAIRE PRIVEE,ST JOSEPH,PRIVE,0,...,211,0,34,45,47,34,51,80-BRETAGNE-RENNES-ILLE-ET-VILAINE-CHANTEPIE-0...,35135,2020
1423,2020,BRETAGNE,RENNES,ILLE-ET-VILAINE,CHANTELOUP,0350168T,ECOLE PRIMAIRE PUBLIQUE,LUCIE AUBRAC,PUBLIC,0,...,139,0,28,28,22,31,30,80-BRETAGNE-RENNES-ILLE-ET-VILAINE-CHANTELOUP-...,35150,2020


In [60]:
COMMUNES = ["RENNES"]
px.bar(ecoles[ecoles["Commune"].isin(COMMUNES)], x="Dénomination principale", y="Nombre total d'élèves", color="rentrée", title="effectif 1er degré RENNES", barmode="group")

In [61]:
COMMUNES = ["CHANTEPIE", "SAINT-GREGOIRE",  "THORIGNE-FOUILLARD" , "RENNES", "VEZIN-LE-COQUET"]
px.bar(ecoles[ecoles["Commune"].isin(COMMUNES)], x="Dénomination principale", y="Nombre total d'élèves", color="rentrée", title="effectif 1er degré des écoles alimentant Rennes", barmode="group")

In [62]:
for classe in ["CE1", "CE2", "CM1", "CM2"]:
    px.bar(ecoles[(ecoles["Commune"].isin(COMMUNES)) & (ecoles["Dénomination principale"] == "ECOLE ELEMENTAIRE PUBLIQUE")], x="Dénomination principale", y=f"Nombre d'élèves en {classe} hors ULIS", color="rentrée", barmode="group", title=classe).show()

,Rentrée scolaire,Région académique,Académie,Département,Commune,Numéro de l'école,Dénomination principale,Patronyme,Secteur,REP,...,Nombre d'élèves en élémentaire hors ULIS,Nombre d'élèves en ULIS,Nombre d'élèves en CP hors ULIS,Nombre d'élèves en CE1 hors ULIS,Nombre d'élèves en CE2 hors ULIS,Nombre d'élèves en CM1 hors ULIS,Nombre d'élèves en CM2 hors ULIS,tri,Code Postal,rentrée
76,2022,BRETAGNE,RENNES,ILLE-ET-VILAINE,CHANTEPIE,0352521A,ECOLE ELEMENTAIRE PUBLIQUE,LES DEUX RUISSEAUX,PUBLIC,0,...,143,0,29,46,23,24,21,78-BRETAGNE-RENNES-ILLE-ET-VILAINE-CHANTEPIE-0...,35135,2022
544,2022,BRETAGNE,RENNES,ILLE-ET-VILAINE,SAINT-GREGOIRE,0350483K,ECOLE ELEMENTAIRE PUBLIQUE,PAUL-EMILE VICTOR,PUBLIC,0,...,250,0,44,48,54,49,55,78-BRETAGNE-RENNES-ILLE-ET-VILAINE-SAINT-GREGO...,35760,2022
659,2022,BRETAGNE,RENNES,ILLE-ET-VILAINE,VEZIN-LE-COQUET,0351619V,ECOLE ELEMENTAIRE PUBLIQUE,ERIC TABARLY,PUBLIC,0,...,169,0,35,29,33,35,37,78-BRETAGNE-RENNES-ILLE-ET-VILAINE-VEZIN-LE-CO...,35132,2022
632,2022,BRETAGNE,RENNES,ILLE-ET-VILAINE,THORIGNE-FOUILLARD,0351765D,ECOLE ELEMENTAIRE PUBLIQUE,LES GRANDS PRES VERTS,PUBLIC,0,...,322,0,50,71,81,61,59,78-BRETAGNE-RENNES-ILLE-ET-VILAINE-THORIGNE-FO...,35235,2022
419,2022,BRETAGNE,RENNES,ILLE-ET-VILAINE,RENNES,0350638D,ECOLE ELEMENTAIRE PUBLIQUE,CARLE BAHON,PUBLIC,0,...,168,0,31,30,34,33,40,78-BRETAGNE-RENNES-ILLE-ET-VILAINE-RENNES-0350...,35200,2022
418,2022,BRETAGNE,RENNES,ILLE-ET-VILAINE,RENNES,0350635A,ECOLE ELEMENTAIRE PUBLIQUE,POTERIE,PUBLIC,0,...,205,0,31,39,50,42,43,78-BRETAGNE-RENNES-ILLE-ET-VILAINE-RENNES-0350...,35200,2022
417,2022,BRETAGNE,RENNES,ILLE-ET-VILAINE,RENNES,0350632X,ECOLE ELEMENTAIRE PUBLIQUE,COLOMBIER,PUBLIC,0,...,140,0,24,33,29,32,22,78-BRETAGNE-RENNES-ILLE-ET-VILAINE-RENNES-0350...,35000,2022
416,2022,BRETAGNE,RENNES,ILLE-ET-VILAINE,RENNES,0350631W,ECOLE ELEMENTAIRE PUBLIQUE,MOULIN DU COMTE,PUBLIC,0,...,234,0,58,46,48,39,43,78-BRETAGNE-RENNES-ILLE-ET-VILAINE-RENNES-0350...,35000,2022
415,2022,BRETAGNE,RENNES,ILLE-ET-VILAINE,RENNES,0350627S,ECOLE ELEMENTAIRE PUBLIQUE,DUCHESSE ANNE,PUBLIC,0,...,168,5,36,23,43,41,25,78-BRETAGNE-RENNES-ILLE-ET-VILAINE-RENNES-0350...,35000,2022
414,2022,BRETAGNE,RENNES,ILLE-ET-VILAINE,RENNES,0350626R,ECOLE ELEMENTAIRE PUBLIQUE,CAMILLE CLAUDEL,PUBLIC,0,...,121,0,21,25,22,26,27,78-BRETAGNE-RENNES-ILLE-ET-VILAINE-RENNES-0350...,35700,2022


In [83]:
colonnes = [f"Nombre d'élèves en {classe} hors ULIS" for classe in ["CE1", "CE2", "CM1", "CM2"]]
# seulement l'année 2022
COMMUNES = ["CHANTEPIE", "SAINT-GREGOIRE",  "THORIGNE-FOUILLARD" , "RENNES", "VEZIN-LE-COQUET"]

ecoles_2022 = ecoles[(ecoles["Commune"].isin(COMMUNES)) & (ecoles["Dénomination principale"] == "ECOLE ELEMENTAIRE PUBLIQUE") & (ecoles["rentrée"] == "2022")]
effectifs_montant = ecoles_2022[colonnes].melt(var_name="niveau", value_name="effectifs").groupby("niveau").sum()
px.bar(effectifs_montant, title="effectifs en 2022 pour les écoles primaires pubiques des communes alimentant Rennes")

In [82]:
effectifs_montant

,effectifs
niveau,
Nombre d'élèves en CE1 hors ULIS,1624
Nombre d'élèves en CE2 hors ULIS,1695
Nombre d'élèves en CM1 hors ULIS,1533
Nombre d'élèves en CM2 hors ULIS,1545
